In [7]:
from sqlalchemy import create_engine
import pymysql
# pip install apache-airflow
# import airflow as arf
# import psycopg2


In [23]:
import pandas as pd
import numpy as np
from sqlite3 import connect
import matplotlib.pyplot as plt

paises_migraciones_url='https://raw.githubusercontent.com/elgualas/datallake_PG/main/migration%20(1).csv'       
paises_poblacion_url='https://raw.githubusercontent.com/elgualas/datallake_PG/main/population-and-demography.csv'

paises_migraciones= pd.read_csv(paises_migraciones_url)
paises_poblacion = pd.read_csv(paises_poblacion_url)

conn = connect(':memory:') 
paises_migraciones.to_sql('pam',conn,index=False)
paises_poblacion.to_sql('pap',conn,index=False)

paises_migraciones_america=pd.read_sql('SELECT Year,Country,Emigrants,`International migrants`, `Net migration`, `Net migration rate` FROM pam WHERE Country IN ("Argentina","Belice","Bolivia", "Brasil","Canada","Colombia", "Costa Rica", "Cuba", "Chile", "Ecuador", "El Salvador", "Guatemala", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay", "Peru","Puerto Rico", "Republica Dominicana","United States","Uruguay","Venezuela","Antigua y Barbuda","Bahamas","Barbados","Dominica","Granada","Guyana","Jamaica","República Dominicana","San Cristobal y Nieves","San Vicente y las Granadinas","Santa Lucia")',conn)
paises_poblacion_america=pd.read_sql('SELECT `Country name`, Year, Population FROM pap WHERE `Country name` IN ("Argentina","Bolivia","Belice", "Brasil","Canada","Colombia", "Costa Rica", "Cuba", "Chile", "Ecuador", "El Salvador", "Guatemala", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay", "Peru","Puerto Rico", "Republica Dominicana","United States","Uruguay","Venezuela","Antigua y Barbuda","Bahamas","Barbados","Dominica","Granada","Guyana","Jamaica","República Dominicana","San Cristobal y Nieves","San Vicente y las Granadinas","Santa Lucia")',conn)

paises_migraciones_america.to_sql('a',conn,index=False)
paises_poblacion_america.to_sql('c',conn,index=False)

tabla=pd.read_sql('SELECT a.Year,a.Country,a.Emigrants,a.`International migrants`,a.`Net migration`,a.`Net migration rate`,c.Population FROM a a JOIN c c ON(a.Year=c.Year AND a.`Country`=c.`Country name` )',conn)

tabla.dropna(inplace=True)
tabla.reset_index(drop=True, inplace=True)

tabla['relative migration']=0

for i in range(len(tabla)):
    tabla['relative migration'][i]=tabla['Emigrants'][i]/tabla['Population'][i]


print(tabla)

C:\Users\aleja\AppData\Local\Temp\ipykernel_10600\2561463446.py:30 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

     Year    Country  Emigrants  International migrants  Net migration  \
0    1990  Argentina   430155.0               1649919.0       160000.0   
1    1995  Argentina   481968.0               1595069.0      -105000.0   
2    2000  Argentina   555542.0               1540219.0      -128000.0   
3    2005  Argentina   805291.0               1674163.0      -125000.0   
4    2010  Argentina   938639.0               1805957.0      -120000.0   
..    ...        ...        ...                     ...            ...   
170  2000  Venezuela   317753.0               1013738.0        -4540.0   
171  2005  Venezuela   437087.0               1076474.0       -22740.0   
172  2010  Venezuela   560528.0               1347347.0      -200220.0   
173  2015  Venezuela   709140.0               1404448.0      -431040.0   
174  2020  Venezuela  5415337.0               1324193.0     -3266243.0   

     Net migration rate  Population  relative migration  
0                 1.019    32637658            0.0131

In [25]:
tabla

,Year,Country,Emigrants,International migrants,Net migration,Net migration rate,Population,relative migration
0,1990,Argentina,430155.0,1649919.0,160000.0,1.019,32637658,0.013180
1,1995,Argentina,481968.0,1595069.0,-105000.0,-0.623,34946116,0.013792
2,2000,Argentina,555542.0,1540219.0,-128000.0,-0.714,37070772,0.014986
3,2005,Argentina,805291.0,1674163.0,-125000.0,-0.660,39070504,0.020611
4,2010,Argentina,938639.0,1805957.0,-120000.0,-0.602,41100124,0.022838
...,...,...,...,...,...,...,...,...
170,2000,Venezuela,317753.0,1013738.0,-4540.0,-0.039,24427732,0.013008
171,2005,Venezuela,437087.0,1076474.0,-22740.0,-0.180,26668790,0.016389
172,2010,Venezuela,560528.0,1347347.0,-200220.0,-1.460,28715026,0.019520
173,2015,Venezuela,709140.0,1404448.0,-431040.0,-2.946,30529722,0.023228


In [20]:
# Create a connection string:
# Replace the values with your RDS database information
DATABASE_TYPE = 'mysql+pymysql'
USERNAME = 'admin'
PASSWORD = '12345678'
HOST = 'database-1.cqu1nfxhhu5u.us-west-2.rds.amazonaws.com'
PORT = '3306'
DATABASE_NAME = 'migration_flow'

connection_string = f"{DATABASE_TYPE}://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}"

In [21]:
# Create an engine object:
engine = create_engine(connection_string)


In [22]:
# Execute a query
result_set = engine.execute('SELECT * FROM prueba')
for row in result_set:
    print(row)



(1, 1999)


In [16]:
#loading dataframes into rds proccese
df = tabla
table_name = "your-table-name"
df.to_sql(table_name, engine, if_exists="replace", index=False)


{'database-1.cqu1nfxhhu5u.us-west-2.rds.amazonaws.com'}

Establish a connection to your RDS instance: You will need to create a connection object that contains the necessary parameters to connect to your RDS instance. You will need to provide the following information: the database endpoint (usually a URL), the database name, the username, and the password.

In [18]:

conn = psycopg2.connect(
    host='database-1.cqu1nfxhhu5u.us-west-2.rds.amazonaws.com',
    database='migration_flow',
    user='admin',
    password='12345678')

OperationalError: connection to server at "database-1.cqu1nfxhhu5u.us-west-2.rds.amazonaws.com" (52.32.215.8), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?
